# Прикладная статистика. ДЗ 1.
# Академия Аналитиков Авито

__Правила:__
- Финальный дедлайн: **2022-11-30 23:59**. 
- После того как ваше решение проверят и ответят, вам дается неделя на исправление тех задач, о которых скажет проверяющий. Ответ и обсуждение решения — в телеграме.

- Выполненную работу нужно отправить
    - в чатик HW1-<ваше имя> через бота @AAA_stats23_bot
    - или в личные сообщения боту.
- В качестве решения нужно отправить файл ipynb. Ссылка на интернет-ресурсы не принимается. Не публикуйте решения в открытом доступе!
- Для выполнения задания используйте этот ноутбук в качествие основы, ничего не удаляя из него. **При этом можно добавлять новые ячейки!**
- в ячейках с комменарием `#Автопроверка` нужно заполнить содержимое функций и классов (если есть), которые будут уже объявлены в этой ячейке. При этом:
    - Нельзя убрирать или переставять `#Автопроверка` в ячейке. 
    - Нельзя менять сигнатуру и возвращаемое значение функций. То есть добавлять любой код можно, но удалять, что уже написано - нельзя.
    - Нельзя ничего импортировать в таких ячейках. Все доступные для использования библиотеки будут указаны заранее. Такие слова, как `import`, `globals`, `locals`, `eval`, `exec` также нельзя использовать внутри ячеек.
    - Нельзя использовать библиотеки, кроме тех, что указаны в задании. Ваш код должен работать именно с эти набором библиотек без любого дополнительного импорта!
    - Нельзя использовать код из других ячеек ноутбука (кроме ячейки с импортом, в которой указаны все доступные библиотеки). Единственное исключение - если вы проставите в начало такой ячейки слово `#Автопроверка`. Тогда вы можете использовать код из этой ячейки.
    - В случае нарушения этого правила автопроверка будет провалена и вы не получите часть баллов за задачу. 
    - В случае, если есть несколько ячеек автопроверки, то в каждой такой ячейке можно использовать созданные вами функции (или классы) из других ячеек автопроверки.

In [2]:
from collections import namedtuple
from scipy.stats import binom
import math
import numpy as np

# Общие замечания по задачам с автопроверкой

Во всех задачах далее, где потребуется автопроверка, можно использовать только следующие библиотеки:

```
from collections import namedtuple
from scipy.stats import binom
import math
import numpy as np
```

Еще раз обращаем внимание, что в ячейках автопроверки __не__ должны быть импортированы какие-либо библиотеки. В других ячейках импортировать библиотеки можно, но при проверке использованы будут только указанные.

## Задача 1. 3 балла

[Осьминог Пауль](https://www.championat.com/football/article-3238881-samye-izvestnye-zhivotnye-predskazateli.html) 14 раз пробовал угадать победителя футбольного матча.
12 раз он угадал правильно, 2 раза — ошибся. Мы хотим проверить гипотезу:

 - $H_0$: осьминог угадывает победителя матча с вероятностью 0.5
 - $H_1$: осьминог выбирает победителя  матча с вероятностью $\neq$ 0.5

#### Пункт А. 1 балл: 

### На зачет

Вам нужно разработать статистический критерий для проверки этой гипотезы, а также посчитать p-value.

- Рассмотрим статистику  $T(X^n) = \underset{i=1}{\overset{n}{\sum}} X_i,\ T \overset{H_0}{\sim} \text{Binom} (n, p_0)$, где $X_i = \begin{cases}
                   1,  & \mbox{если осьминог угадал}\\
                   0, & \mbox{если осьминог не угадал}
               \end{cases}.$
               
- Когда мы отвергнем поставленную гипотезу?
    - Пусть реализация $T(X^n) = t$. Тогда если $P_{H_0}(T(X^n) \leq t) \leq \alpha / 2$ и $P_{H_0}(T(X^n) \geq t) \leq \alpha/2$ то мы отвергнем гипотезу.  
       $
        \begin{align}
            &t \geq F^{-1}(1 - \alpha/2) + 1 \\
            &t \leq F^{-1}(\alpha/2)
        \end{align}
        $

Реализуйте критерий `check_paul_criterion(n, k, alpha)`, где 
- `n` &mdash; количество матчей;
- `k` &mdash; количество верных предсказаний от осьминога;
- `alpha` &mdash; уровень значимости критерия.

Функция должна вернуть `PaulCheckResults` с полями
- is_rejected: отверглась или нет гипотеза H_0 на уровне значимости alpha
- pvalue


In [3]:
# Автопроверка

PaulCheckResults = namedtuple('PaulCheckResults', ['is_rejected', 'pvalue'])

def check_paul_criterion(n: int, k: int, alpha: float=0.05, two_side: bool=True):
    """
    Параметры:
    - n: количество матчей
    - k: количество верных предсказаний от осьминога
    - alpha: уровень значимости критерия.
    - two_side: использовать или нет двустороннюю гипотезу
        
    Возвращает:
    - PaulCheckResults с полями:
        - is_rejected: bool
            - отверглась или нет гипотеза H_0 
                       на уровне значимости alpha
        - pvalue: float
    """
    
    critical_value_right = binom.ppf(1 - alpha / 2, n=n, p=0.5) + 1
    critical_value_left = binom.ppf(alpha / 2, n=n, p=0.5) - 1

    is_rejected = (k <= critical_value_left) or (critical_value_right <= k)

    pvalue_onesided = 1 - binom.cdf(k - 1, n=n, p=0.5)

    if pvalue_onesided > 0.5:
        pvalue_onesided = binom.cdf(k, n=n, p=0.5)

    pvalue_twosided = 2 * pvalue_onesided

    if pvalue_twosided > 1:
        pvalue_twosided = 1
    pvalue = pvalue_twosided

    return PaulCheckResults(is_rejected, pvalue)

In [4]:
res = check_paul_criterion(14, 12)
res.is_rejected, res.pvalue

(True, 0.012939453125)

#### Пункт B. 2 балла: 

Какие выводы можно сделать из полученного результата?

Так как осьминоги не учавствуют в программе экстрасенсы, то очень вероятно, что таких способностей они не имеют, скорее всего выборка из 14 испытаний слишком маленькая. Плюс так как 12 из 14 испытаний он угадал, можно попробовать рассматривать односторонний критерий


In [43]:
import scipy.stats as sps 


def get_critical_values(p0, n, alpha=0.05):
    if alpha >= 0.5:
        raise ValueError('Alpha has to be less then 0.5')
        
    critical_value_right = binom.ppf(1 - alpha / 2, n=n, p=0.5) + 1
    critical_value_left = binom.ppf(alpha / 2, n=n, p=0.5) - 1
    return critical_value_left, critical_value_right

def get_mde(p0, n, power, 
            critical_value_left = None, 
            critical_value_right = None, 
            is_reverse_order = False):
    mde = 100
    
    if not is_reverse_order:
        possible_p = np.arange(p0, 1.001, 0.001)
    else:
        possible_p = np.arange(p0 + 0.001, -0.001, -0.001)
    
    right_area, left_area = 0, 0
    for p in possible_p:   
        
        if critical_value_right is not None:
            right_area = 1 - sps.binom(p=p, n=n).cdf(critical_value_right - 1) 
            
        if critical_value_left:
            left_area = sps.binom(p=p, n=n).cdf(critical_value_left)
            
        if right_area + left_area >= power:
            mde = round((p - p0) * 100, 2) if not is_reverse_order else round((p0 - p) * 100, 2)
            break
            
    return mde

def get_interval_and_MDE(p0, n, power, 
                         critical_value_left = None, 
                         critical_value_right = None):
    
    mde = None
    left_mde, right_mde = 0, 0
    
    if critical_value_right is not None:
        right_mde = get_mde(p0, n, power, 
                            critical_value_left, critical_value_right, 
                            is_reverse_order=False)
        mde = right_mde
        interval = round(p0 + right_mde / 100, 2)
        
    if critical_value_right is not None:
        left_mde = get_mde(p0, n, power, 
                           critical_value_left, critical_value_right, 
                           is_reverse_order=True)
        if mde is not None:
            mde += mde
            interval = (round(p0 - left_mde / 100, 2), interval)
        else:
            mde = right_mde
            interval = round(p0 - left_mde / 100, 2)
            
    return {'interval': interval, 'MDE': mde}

In [54]:
p0 = 0.5
alpha=0.05
n = 14
k = 12
power = 0.8

critical_value_left, critical_value_right = get_critical_values(p0, n, alpha)

get_interval_and_MDE(p0, n, power, critical_value_left, critical_value_right)

{'interval': (0.11, 0.89), 'MDE': 77.8}

При мощности 80%(нас она устраивает) доверительный интервал слишком большой(такой нас не утсраивает), так как мы можем задетектировать только, если истинная вероятность будет сильно отличаться. Поэтому, как вариант нужно увеличить количество наблюдений, наример пусть n=3500 MDE будет меньше 5%:

In [63]:
p0 = 0.5
alpha=0.05
n = 3500
k = 12
power=0.8

critical_value_left, critical_value_right = get_critical_values(p0, n, alpha)

get_interval_and_MDE(p0, n, power, critical_value_left, critical_value_right)

{'interval': (0.48, 0.52), 'MDE': 4.8}

## Задача 2. 3 балла

### На зачет

Мы разработали новый дизайн нашего продукта. Вероятность, что он понравится случайному человеку — $p$, и она нам неизвестна. Мы хотим
проверить $H_0: p = 1$ с помощью статистического критерия c уровнем значимости $\alpha$. Предложить критерий для решения этой задачи.

- Пусть $p$ &mdash; истинная вероятность того, что пользователю понравится дизайн. Тогда наша гипотеза звучит так: 
    - $H_0: p = 1.0 \ vs.\ H_1: p < 1.0$ 
    - Пусть ошибка первого рода 5%. Но есть нюанс, из за распределения для гипотезы H0 изменение $\alpha$ не будет влиять на результат
- Какой критерий использовать?
    - Рассмотрим статистику  $T(X^n) = \underset{i=1}{\overset{n}{\sum}} X_i,\ T \overset{H_0}{\sim} \text{Binom} (n, p_0)$, где $X_i = \begin{cases}
                   1,  & \mbox{если человеку понравилось}\\
                   0, & \mbox{если человеку не понравилось}
               \end{cases},$ $n$ $\in$ [0, 1000].
    - Когда мы отвергнем поставленную гипотезу?
        - Пусть реализация $T(X^n) = t$. Тогда если $P_{H_0}(T(X^n) \leq t) \leq \alpha$, то мы отвергнем гипотезу.
        $$
        \begin{align}
            &P_{H_0}(T(X^n) \leq t) \leq \alpha \Leftrightarrow\\
            &F_{\text{Binom}(n, p_0)}(t) \leq \alpha \Leftrightarrow\\
            &t \leq F^{-1}(\alpha)
        \end{align}
        $$ => для любой $\alpha$ для $H_0$ будет $t = n*(n+1)/2 - 1$
        
    Чтобы найти начало крит. области, найдо найти мин t, чтобы $F(x) \geq p$. Вспомним определение квантили:  
    $u_p = \{\min\ x: F(x) \geq p \}$. Тогда это и есть искомое значение: начало критической области.
    - `reject_value = binom(p=p0, n=n).ppf(alpha) - 1`. У нас есть правило для отвержения гипотезы  
    => reject_value = $n-1$
    
- Мы знаем, что нулевая гипотеза отвергается, если $T \leq \text{reject_value}$
- Тогда нам надо найти такое $p$, чтобы $P_{\text{Binom} (n, p)}(T(X^n) \leq \text{reject_value}) = 1-\beta$:
    $$\begin{align}
    &P_{\text{Binom} (n, p)}(T(X^n) \leq \text{reject_value}) \geq 1-\beta \Leftrightarrow\\
    &1 - P_{\text{Binom} (n, p)}(T(X^n) \leq \text{reject_value}) \leq \beta
    \end{align}
    $$
    То есть мы ищем такое p, что `1 - binom(p=p, n=N).cdf(reject_value) <= beta`

Напишите функцию `calculate_number_of_users(alpha, beta, p)` — скольки людям нужно показать этот дизайн, чтобы добиться мощности `1 - beta` при заданном `p` и уровне значимости `alpha`.

P.S. Утверждается, что на наших тестах ответ не будет превосходить 1000 человек.

In [4]:
#Автопроверка

def calculate_number_of_users(alpha: float, beta: float, p: float):
    """
    Параметры:
    - alpha: уровень значимости
    - beta: инвертированная мощность критерия. мощность = 1 - beta.
    - p: истинная вероятность того, что пользователю понравится дизайн.
    Возвращает:
    - number_of_users: int
        - количество людей, которым надо показать дизайн.
    """
    p0 = 1
    number_of_users = None

    for n in range(1, 1001):
    
        reject_value = binom(p=p0, n=n).ppf(alpha) - 1
        
        if 1 - binom(p=p, n=n).cdf(reject_value) <= beta:
            number_of_users = n
            break   

    return number_of_users

In [5]:
calculate_number_of_users(0.05, 0.3, 0.8)

6

## Задача 3.

По недостоверной информации (вероятность, что она верна, считаем за 1%), в новой версии нашего сайта есть сложнодетектируемый баг. Мы могли бы попросить разработку его отыскать и починить, но на это уйдет много ресурсов.

К счастью, у нас есть старый AB тест (новая версия сайта vs старая), который мы можем проанализировать и с некоторой вероятностью обнаружить наличие бага просто сравнением выборок. У нас есть три критерия для проверки гипотезы "$H_0$: баги нет, $H_1$: баг есть":
- критерий `A`: $\alpha = 0.02, 1-\beta = 0.50$
- критерий `B`: $\alpha = 0.05, 1-\beta = 0.60$
- критерий `C`: $\alpha = 0.10, 1-\beta = 0.70$

Если критерий находит баг, мы просим разработчиков потратить силы и починить. На это у них уйдет усилий на 1 М ₽ независимо от того, найдут они баг или нет.
Если критерий не найдет баг, затраты разработчиков будут нулевыми, но из-за бага мы потеряем в конечном итоге 50 М ₽.

#### Пункт А. 2 балла: 

Какой критерий стоит выбрать?

Матожидание потерь, если критерий нашел баг: Баг есть + Бага нет: $$(1 - \beta)* 0.01 * 50млн + \alpha * 0.99 * 50млн$$ 
Матожидание потерь, если критерий не нашел баг: Баг есть + Бага нет: $$0.01*(1 - (1 - \beta)) * 1млн + 0.99*(1 - \alpha) * 0млн$$

In [6]:
def E(alpha, betta, dev_money=1e6, lost_money=50e6, p_bag=0.01):
    M_find_bag = (1 - betta) * p_bag * dev_money + alpha * (1 - p_bag) * dev_money
    M_not_find_bag = betta * p_bag * lost_money + (1 - alpha) * (1 - p_bag) * 0
    return M_find_bag + M_not_find_bag

In [7]:
criteria = [('A', 0.02, 1 - 0.50), ('B', 0.05, 1 - 0.60), ('C', 0.1, 1 - 0.70)]
for c, alpha, beta in criteria:
    print(c , ': ', round(E(alpha, beta)))

A :  274800
B :  255500
C :  256000


#### Выбираю критерий B

#### Пункт B. 2 балла: 
Предложите оптимальную стратегию, если потери от ненайденного бага составят вместо 50М:
- 20М рублей;
- 3М рублей;
- 300М рублей.

In [8]:
lost_moneys = [20e6, 3e6, 300e6]
for lost_money in lost_moneys:
    for c, alpha, beta in criteria:
        print(c , ': ', round(E(alpha, beta, 1e6, lost_money)))
    print()

A :  124800
B :  135500
C :  166000

A :  39800
B :  67500
C :  115000

A :  1524800
B :  1255500
C :  1006000



Матожидание потерь, что баг есть для каждого случая:
- 20М рублей:  0.01 * 20000000 = 200000 > 124800 
- 3М рублей:  0.01 * 3000000 = 30000 < 39800 
- 300М рублей:  0.01 * 300000000 = 3000000 > 1006000  

Получается, что для случая 3млн нам невыгодно искать баг, так как матожидание потерь случая, когда мы его не ищем, меньше чем матожидание потерь в случае, когда мы пытаемся найти баг.

Вывод:
Для 20млн ищем баг и выбираем критерий А, для 3млн нам выгоднее не искать баг вовсе, для 300млн критерий С
